In [8]:
import random
import pandas as pd
import numpy as np
import tensorflow as tf
from tensorflow import keras
import time
import matplotlib.pyplot as plt

X_column = [ '日付(hour)','日付(minute)', 'ドル円', 'ポンド円', 'オシレーター買いドル円', 'オシレーター中立ドル円', 'オシレーター売りドル円',
       '移動平均買いドル円', '移動平均中立ドル円', '移動平均売りドル円', 'オシレーター買いポンド円', 'オシレーター中立ポンド円',
       'オシレーター売りポンド円', '移動平均買いポンド円', '移動平均中立ポンド円', '移動平均売りポンド円', 'S&P500指数',
       '日経平均株価', 'UK100INDEX','恐怖指数']

# csvデータ整形
def data_shape():
    #------------csvからゴミデータを削除---------------------------
    df= pd.read_csv("為替情報.csv",encoding='shift_jis',index_col=0)
    #条件にマッチしたIndexを取得
    drop_index = df.index[df['ドル円(15分後)'] == 0]
    #条件にマッチしたIndexを削除
    df = df.drop(drop_index)
    df = df.dropna()  # NaNを削除
    df.to_csv('FX再現用pd自動整形.csv', encoding='shift_jis')
    #------------csvからゴミデータを削除---------------------------
    df= pd.read_csv("FX再現用pd自動整形.csv",encoding='shift_jis')
    print(df)
    y_usdjpy_train = df['ドル円(15分後)']  # 目的変数
    y_gbpjpy_train = df['ポンド円(15分後)']  # 目的変数
    df['日付'] = pd.to_datetime(df['日付'])  # 日付カラムを日付型に変換
    df['日付(hour)'] = df['日付'].dt.hour  # hourをデータに追加
    df['日付(minute)'] = df['日付'].dt.minute  # minuteをデータに追加
    X = df[X_column]  # 特徴量
    return df, X, y_usdjpy_train, y_gbpjpy_train

In [9]:
df, X, y_usdjpy_train, y_gbpjpy_train = data_shape()  # データ整形
X_train = X  # トレーニングデータの作成
print(df.dtypes)




                        日付      ドル円     ポンド円  オシレーター買いドル円  オシレーター中立ドル円  \
0     2020/12/17/ 06:24:11  103.481  139.650          2.0          7.0   
1     2020/12/17/ 06:25:45  103.483  139.661          2.0          7.0   
2     2020/12/17/ 06:27:20  103.480  139.698          2.0          7.0   
3     2020/12/17/ 06:28:54  103.470  139.674          2.0          7.0   
4     2020/12/17/ 06:30:29  103.474  139.683          2.0          7.0   
...                    ...      ...      ...          ...          ...   
7786  2020/12/30/ 00:48:04  103.529  139.808          1.0          8.0   
7787  2020/12/30/ 00:49:38  103.550  139.765          1.0          8.0   
7788  2020/12/30/ 00:51:13  103.527  139.752          1.0          8.0   
7789  2020/12/30/ 00:52:47  103.538  139.737          1.0          8.0   
7790  2020/12/30/ 00:54:21  103.555  139.729          1.0          8.0   

      オシレーター売りドル円  移動平均買いドル円  移動平均中立ドル円  移動平均売りドル円  オシレーター買いポンド円  ...  \
0             2.0        0.0        1.

In [14]:
def model_create():
    # ニューラルネットワーク作成
    n_inputs = len(X_column)  # 入力数
    print('入力数', n_inputs)
    model = keras.models.Sequential([
        keras.layers.Dense(12, activation='relu', input_shape=[n_inputs]),  # n_inputsを入力して隠れユニットを出す
        keras.layers.Dense(6, activation='relu'),
        keras.layers.Dense(1, activation='linear')
    ])
    optimizer = keras.optimizers.Adam(lr=0.1)  # オプティマイザ
    loss_fn = keras.losses.mse  # 損失関数
    model.compile(loss=loss_fn, optimizer=optimizer, metrics=['mae'])  # コンパイル
    model.summary()
    return model

In [15]:
model_usdjpy = model_create()
model_gbpjpy = model_create()

入力数 20
Model: "sequential_4"
_________________________________________________________________
Layer (type)                 Output Shape              Param #   
dense_12 (Dense)             (None, 12)                252       
_________________________________________________________________
dense_13 (Dense)             (None, 6)                 78        
_________________________________________________________________
dense_14 (Dense)             (None, 1)                 7         
Total params: 337
Trainable params: 337
Non-trainable params: 0
_________________________________________________________________
入力数 20
Model: "sequential_5"
_________________________________________________________________
Layer (type)                 Output Shape              Param #   
dense_15 (Dense)             (None, 12)                252       
_________________________________________________________________
dense_16 (Dense)             (None, 6)                 78        
____________________

In [16]:
print('X_train：',X_train.shape)
print('y_usdjpy_train：',y_usdjpy_train.shape)
print('y_gbpjpy_train：',y_gbpjpy_train.shape)


X_train： (7791, 20)
y_usdjpy_train： (7791,)
y_gbpjpy_train： (7791,)


In [17]:
# 予測 USDJPY
history_USDJPY = model_usdjpy.fit(X_train, y_usdjpy_train, batch_size=1, epochs=30,validation_split=0.05)

Epoch 1/30
7401/7401 [==============================] - 6s 704us/step - loss: 14286.9491 - mae: 49.2449 - val_loss: 0.0381 - val_mae: 0.1910
Epoch 2/30
7401/7401 [==============================] - 5s 676us/step - loss: 0.0342 - mae: 0.1466 - val_loss: 0.0208 - val_mae: 0.1386
Epoch 3/30
7401/7401 [==============================] - 8s 1ms/step - loss: 0.0401 - mae: 0.1589 - val_loss: 0.0043 - val_mae: 0.0595
Epoch 4/30
7401/7401 [==============================] - 7s 1ms/step - loss: 0.0402 - mae: 0.1589 - val_loss: 0.0309 - val_mae: 0.1712
Epoch 5/30
7401/7401 [==============================] - 7s 993us/step - loss: 0.0416 - mae: 0.1624 - val_loss: 0.0295 - val_mae: 0.1670
Epoch 6/30
7401/7401 [==============================] - 7s 976us/step - loss: 0.0405 - mae: 0.1593 - val_loss: 0.0070 - val_mae: 0.0736
Epoch 7/30
7401/7401 [==============================] - 7s 1ms/step - loss: 0.0410 - mae: 0.1596 - val_loss: 0.0592 - val_mae: 0.2399
Epoch 8/30
7401/7401 [===========================

In [18]:
# 予測 GBPJPY
history_GBPJPY = model_gbpjpy.fit(X_train, y_gbpjpy_train, batch_size=1, epochs=30,validation_split=0.05)

Epoch 1/30
7401/7401 [==============================] - 9s 1ms/step - loss: 132940.8962 - mae: 36.8525 - val_loss: 13.3967 - val_mae: 3.6394
Epoch 2/30
7401/7401 [==============================] - 9s 1ms/step - loss: 231.7174 - mae: 6.7945 - val_loss: 0.1286 - val_mae: 0.3418
Epoch 3/30
7401/7401 [==============================] - 8s 1ms/step - loss: 0.7211 - mae: 0.7135 - val_loss: 0.3688 - val_mae: 0.5976
Epoch 4/30
7401/7401 [==============================] - 8s 1ms/step - loss: 0.7482 - mae: 0.7210 - val_loss: 0.1748 - val_mae: 0.4039
Epoch 5/30
7401/7401 [==============================] - 9s 1ms/step - loss: 0.7492 - mae: 0.7157 - val_loss: 0.1776 - val_mae: 0.4073
Epoch 6/30
7401/7401 [==============================] - 9s 1ms/step - loss: 0.7810 - mae: 0.7412 - val_loss: 0.1855 - val_mae: 0.4168
Epoch 7/30
7401/7401 [==============================] - 8s 1ms/step - loss: 0.7756 - mae: 0.7325 - val_loss: 0.2797 - val_mae: 0.5176
Epoch 8/30
7401/7401 [==============================]

In [ ]:
pd.DataFrame(history.history).plot(figsize=(8, 5))
plt.grid(True)
plt.gca().set_ylim(0, 1)
plt.show()

In [ ]:
X_new = X_train[:1000]
y_new_USDJPY = y_usdjpy_train[:100]
y_new_GBPJPY = y_gbpjpy_train[:100]

In [ ]:
y_pred = model.predict(X_new)

In [ ]:
print(y_pred)
print( y_new)
